In [1]:
import requests
import numpy as np
import pandas as pd
import re
import datetime
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## Analise de FII (Fundo Imobiliário)

In [2]:
def tratar_dados(string):
    try:
        return re.sub(',', '.', re.sub(r'[R$ %\.]*', '', string, flags=re.I))
    except:
        return 0

In [123]:
def setores_e_tickers():
    url = 'https://www.fundsexplorer.com.br/ranking'
    r = requests.get(url)
    df = pd.read_html(r.content, encoding='utf-8')[0]
    df.fillna(value=0, inplace=True)
    setores = df['Setor'].unique().tolist()
    del(setores[setores.index(0)])
    tickers = df['Códigodo fundo'].unique().tolist()
    return setores, tickers

In [124]:
def tabela_indicadores(setor):
    url = 'https://www.fundsexplorer.com.br/ranking'
    r = requests.get(url)
    df = pd.read_html(r.content, encoding='utf-8')[0]
    df.fillna(value=0, inplace=True)
    df_col = df.iloc[:, :2]
    df_cat = df.drop(columns=['Códigodo fundo', 'Setor']).select_dtypes(include='object').applymap(tratar_dados)
    df_num = df.select_dtypes(include='number')
    df = pd.concat([df_col, df_cat, df_num], axis=1)
    df['Códigodo fundo'] = df['Códigodo fundo'].apply(lambda x: f"{x}.SA")
    df.iloc[:, 2:] = df.iloc[:, 2:].astype('float')
    df = df[df['Setor'] != 0]
    df['P/VPA'] = df['P/VPA'] / 100
    indicadores = [
        'Códigodo fundo',
        'Setor',
        'Preço Atual',
        'Dividendo',
        'DY (3M)Acumulado', 
        'DY (6M)Acumulado', 
        'DY (12M)Acumulado',
        'P/VPA',
        'QuantidadeAtivos',
        'Liquidez Diária',
        'VacânciaFísica',
        'VacânciaFinanceira', 
                  ]
    df = df[indicadores]
    return oportunidades(df, setor)

In [41]:
def oportunidades(df, setor=['Shoppings']):
    try:
        df_setor = df[df['Setor'] == setor]
        filter_ = \
            (df_setor['Liquidez Diária'] > 20000) &\
            (df_setor['P/VPA'] >= 0.9) &\
            (df_setor['P/VPA'] <= 1.1) &\
            (df_setor['DY (3M)Acumulado'] > df_setor['DY (3M)Acumulado'].mean()) &\
            (df_setor['VacânciaFísica'] <= 20)
        return df_setor.loc[filter_].sort_values('DY (3M)Acumulado', ascending=False).reset_index(drop=True)
    except Exception as e:
        print(e)

In [42]:
def pegar_fundo(ticker):
    end = datetime.datetime.today()
    start = datetime.datetime(end.year - 1, 1, 1)
    stock = yf.download(ticker, start=start, end=end)
    return stock

In [98]:
def pegar_12m_fundo(ticker):
    try:
        url = f'https://fiis.com.br/{ticker}/'
        r = requests.get(url)
        df = pd.read_html(r.content, encoding='utf-8')[1]
        df['DataPagamento'] = pd.to_datetime(df['DataPagamento'], format='%d/%m/%y')
        df['DataBase'] = pd.to_datetime(df['DataBase'], format='%d/%m/%y')
        df.iloc[:, 2:] = df.iloc[:, 2:].applymap(tratar_dados).astype('float')
        df['% change cotação'] = df['CotaçãoBase'].pct_change(periods=-1)
        df['% change Rendimento'] = df['Rendimento'].pct_change(periods=-1)
        df = df.sort_values('DataPagamento', ascending=True)
        df.fillna(0, inplace=True)
        return df.reset_index(drop=True)
    except:
        return "Ticker não disponível"

In [127]:
labels = setores_e_tickers()
tickers = sorted(labels[1])
setores = sorted(labels[0])

In [130]:
# Ao selecionar o setor, os dados mudarão automaticamente baseado na estratégia definida da função oportunidades.
interact(tabela_indicadores, setor=setores)

interactive(children=(Dropdown(description='setor', options=('Hospital', 'Hotel', 'Híbrido', 'Lajes Corporativ…

<function __main__.tabela_indicadores(setor)>

In [129]:
# basta selecionar o código do fundo para buscar automaticamente o histórico do ticker.
interact(pegar_12m_fundo, ticker=tickers)

interactive(children=(Dropdown(description='ticker', options=('ABCP11', 'AFHI11', 'AFOF11', 'AIEC11', 'ALMI11'…

<function __main__.pegar_12m_fundo(ticker)>